In [6]:
!pip install pandas
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
#imports
import csv
import pandas as pd

### Reading the data

In [8]:
# #read CSV
# rawDataHeader = []
# rawData = []
# with open("eurovision_song_contest_1957_2023.csv") as csvfile:
#     reader = csv.reader(csvfile, delimiter=",")
#     rawDataHeader = next(reader)
#     for row in reader:
#         rawData.append(row)

In [9]:
pathOriginalDataFile = "data/eurovision_song_contest_1957_2023.xlsx"
origDataFrame = pd.read_excel(pathOriginalDataFile)

In [10]:
origDataFrame

,Year,(semi-) final,Edition,Jury or Televoting,From country,To country,Points,Duplicate
0,1957,f,1957f,J,Austria,Austria,0,x
1,1957,f,1957f,J,Austria,Belgium,0,NaN
2,1957,f,1957f,J,Austria,Denmark,0,NaN
3,1957,f,1957f,J,Austria,France,0,NaN
4,1957,f,1957f,J,Austria,Germany,0,NaN
...,...,...,...,...,...,...,...,...
61813,2023,f,2023f,J,United Kingdom,Estonia,6,NaN
61814,2023,f,2023f,J,United Kingdom,Belgium,7,NaN
61815,2023,f,2023f,J,United Kingdom,Lithuania,8,NaN
61816,2023,f,2023f,J,United Kingdom,Australia,10,NaN
